In [15]:
from textwrap import fill
import pandas as pd
import numpy as np

pd.set_option("display.width", 150)
pd.set_option("display.max_rows", 40)

df = pd.DataFrame({
    "season": ["2021/22"]*6 + ["2022/23"]*6,
    "stage": ["Group","Group","Group","Quarter","Quarter","Semi"]*2,
    "team": ["Real Madrid","Man City","Bayern","AC Milan","PSG","Chelsea"]*2,
    "opponent": ["PSG","Chelsea","Liverpool","Real Madrid","Bayern","Man City"]*2,
    "goals_for": [2,3,1,0,2,1, 1,4,2,1,3,2],
    "goals_against": [1,1,3,2,2,0, 2,1,1,1,1,3],
})
print(df.head(10))

df_mi = df.set_index(["season","stage","team"]).sort_index()
print(df_mi)

agg = (df_mi
       .groupby(["season","stage"])
       .agg(
           matches = ("goals_for","count"),
           avg_goals_for = ("goals_for","mean"),
           avg_goals_against = ("goals_against","mean")
       ))
print(agg)

pivot = pd.pivot_table(
    df,
    values="goals_for",
    index=["season","stage"],
    columns="team",
    aggfunc="mean",
    fill_value=0
)

print(pivot)

melted = pivot.reset_index().melt(
    id_vars=["season","stage"],
    var_name="team",
    value_name="avg_goals"
)

print(melted.head(10))

top2 = (melted
        .sort_values(["season","stage","avg_goals"], ascending=[True,True,False])
        .groupby(["season","stage"], group_keys=False)
        .head(2))
print(top2)

df["goal_diff"] = df["goals_for"] - df["goals_against"]

# Avg GF vs GA per (season, stage, team)
goal_difference = (
    df.groupby(["season","stage","team"])
      .agg(
          avg_gf=("goals_for","mean"),
          avg_ga=("goals_against","mean")
      )
      .reset_index()
)
print(goal_difference)

melted.to_csv("ucl_multiindex_analysis.csv", index=False)


    season    stage         team     opponent  goals_for  goals_against
0  2021/22    Group  Real Madrid          PSG          2              1
1  2021/22    Group     Man City      Chelsea          3              1
2  2021/22    Group       Bayern    Liverpool          1              3
3  2021/22  Quarter     AC Milan  Real Madrid          0              2
4  2021/22  Quarter          PSG       Bayern          2              2
5  2021/22     Semi      Chelsea     Man City          1              0
6  2022/23    Group  Real Madrid          PSG          1              2
7  2022/23    Group     Man City      Chelsea          4              1
8  2022/23    Group       Bayern    Liverpool          2              1
9  2022/23  Quarter     AC Milan  Real Madrid          1              1
                                opponent  goals_for  goals_against
season  stage   team                                              
2021/22 Group   Bayern         Liverpool          1              3
       